In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
heroes = {
    25: 'shaman',
    326: 'hunter',
    390: 'warlock',
    494: 'mage',
    612: 'druid',
    754: 'rogue',
    798: 'priest',
    981: 'paladin',
    1235: 'warrior'
}

## Задание 1
Сделать таблицу матчапов 9х9 где по обоим осям названия героев а в ячейке i, j - процент побед(средний decision) героя i против героя j

In [3]:
data = []
with open('data/trainingData_JSON_chunk1.json') as fin:
    for record in map(json.loads, fin):
        data.append((
            record['player']['hero']['hero_card_id'], 
            record['opponent']['hero']['hero_card_id'],
            record['decision'],
        ))
        # Учтём антисимметрию: победы игрока = проигрыши оппонента и наоборот
        data.append((
            record['opponent']['hero']['hero_card_id'],
            record['player']['hero']['hero_card_id'],
            record['decision'] ^ 1,
        ))
games = pd.DataFrame(data, columns=('player1', 'player2', 'decision'))

In [4]:
games['player1'] = games['player1'].map(heroes)
games['player2'] = games['player2'].map(heroes)

In [5]:
games.head()

,player1,player2,decision
0,paladin,priest,1
1,priest,paladin,0
2,rogue,rogue,1
3,rogue,rogue,0
4,druid,druid,1


In [6]:
games.pivot_table('decision', 'player1', 'player2')

player2,druid,hunter,mage,paladin,priest,rogue,shaman,warlock,warrior
player1,,,,,,,,,
druid,0.500000,0.503007,0.496327,0.505086,0.498796,0.508860,0.496575,0.508699,0.514108
hunter,0.496993,0.500000,0.498834,0.496278,0.497046,0.505211,0.497600,0.487393,0.502598
mage,0.503673,0.501166,0.500000,0.499918,0.494756,0.502783,0.489098,0.497063,0.483212
paladin,0.494914,0.503722,0.500082,0.500000,0.519706,0.501785,0.497775,0.498354,0.518996
priest,0.501204,0.502954,0.505244,0.480294,0.500000,0.505109,0.502825,0.496903,0.490903
rogue,0.491140,0.494789,0.497217,0.498215,0.494891,0.500000,0.489657,0.499449,0.491418
shaman,0.503425,0.502400,0.510902,0.502225,0.497175,0.510343,0.500000,0.505697,0.515544
warlock,0.491301,0.512607,0.502937,0.501646,0.503097,0.500551,0.494303,0.500000,0.502785
warrior,0.485892,0.497402,0.516788,0.481004,0.509097,0.508582,0.484456,0.497215,0.500000


## Задание 2
По каждой уникальной карте сделать таблицу с полями:
name	poisonous	shield	stealth	taunt	turn	type	windfury	hero_name
Пример: https://snag.gy/2rFpbz.jpg

In [7]:
def process_card(card, turn, hero):
    def field_or_NaN(field):
        return card[field] if field in card else np.NaN
    return (
        field_or_NaN('name'),
        field_or_NaN('poisonous'),
        field_or_NaN('shield'),
        field_or_NaN('stealth'),
        field_or_NaN('taunt'),
        turn,
        # У сыгранных карт может быть тип только minion
        card['type'] if 'type' in card else 'MINION',
        field_or_NaN('windfury'),
        heroes[hero],
    )
data = set()
with open('data/trainingData_JSON_chunk1.json') as fin:
    for record in map(json.loads, fin):
        # Fixme: Played cards have no 'type' attribute
        for card in record['player']['hand'] + record['player']['played_cards']:
            data.add(process_card(card, record['turn'], record['player']['hero']['hero_card_id']))
        for card in record['opponent']['played_cards']:
            data.add(process_card(card, record['turn'], record['opponent']['hero']['hero_card_id']))
cards = pd.DataFrame(data, columns=('name', 'poisonous', 'shield', 'stealth', 'taunt', 'turn', 'type', 'windfury', 'hero_name'))

In [8]:
cards.head()

,name,poisonous,shield,stealth,taunt,turn,type,windfury,hero_name
0,Gnomish Inventor,False,False,False,False,27,MINION,False,priest
1,Chillwind Yeti,False,False,False,False,5,MINION,False,paladin
2,Chillwind Yeti,False,False,False,True,5,MINION,False,rogue
3,Frog,False,False,False,True,12,MINION,False,warrior
4,Acidic Swamp Ooze,False,False,False,False,22,MINION,False,hunter


In [9]:
# turn не обязательно -- это номер хода 
cards = cards.drop_duplicates(
    ('name', 'poisonous', 'shield', 'stealth', 'taunt', 'type', 'windfury', 'hero_name'))

In [10]:
cards.head()

,name,poisonous,shield,stealth,taunt,turn,type,windfury,hero_name
0,Gnomish Inventor,False,False,False,False,27,MINION,False,priest
1,Chillwind Yeti,False,False,False,False,5,MINION,False,paladin
2,Chillwind Yeti,False,False,False,True,5,MINION,False,rogue
3,Frog,False,False,False,True,12,MINION,False,warrior
4,Acidic Swamp Ooze,False,False,False,False,22,MINION,False,hunter


## Задание 3
Посчитать средний винрейт(средний decision) по каждой карте (среди колод, где данная карта присутствует)

In [11]:
cards_winrate = dict((card, [0, 0]) for card in cards['name'].unique())
with open('data/trainingData_JSON_chunk1.json') as fin:
    for record in map(json.loads, fin):
        for card in record['player']['hand'] + record['player']['played_cards']:
            cards_winrate[card['name']][0] += 1
            cards_winrate[card['name']][1] += record['decision']
        for card in record['opponent']['played_cards']:
            cards_winrate[card['name']][0] += 1
            cards_winrate[card['name']][1] += record['decision'] ^ 1

In [12]:
for name in cards_winrate:
    cards_winrate[name] = cards_winrate[name][1] / cards_winrate[name][0]

In [13]:
cards_winrate

{'Gnomish Inventor': 0.5438387106755604,
 'Chillwind Yeti': 0.5604228371802287,
 'Frog': 0.5524358372981304,
 'Acidic Swamp Ooze': 0.5474572601338745,
 'Bloodfen Raptor': 0.5438082373680716,
 'Murloc Tidehunter': 0.5520228045170485,
 'Silver Hand Recruit': 0.5944298245614035,
 'Wild Growth': 0.49690377897745314,
 'Shattered Sun Cleric': 0.547186456173837,
 'Truesilver Champion': 0.4650992634505634,
 'Sheep': 0.6025688073394495,
 'Boar': 0.5815162809382463,
 'Hellfire': 0.4918275588468659,
 'River Crocolisk': 0.5490901581898278,
 'Frostwolf Warlord': 0.5612045490949864,
 'Fire Elemental': 0.5691359365665563,
 'Flametongue Totem': 0.5146261922860015,
 'Bloodlust': 0.46059684774781984,
 'Healing Totem': 0.5775956544527565,
 'Boulderfist Ogre': 0.5612510702802259,
 'Darkscale Healer': 0.5641767659002974,
 'Kobold Geomancer': 0.5350133341323825,
 'Claw': 0.4831876260928043,
 'Swipe': 0.5100305456947082,
 'Mortal Coil': 0.45874451754385964,
 'Consecration': 0.4884387447780045,
 'Ironbark Pro